<a href="https://colab.research.google.com/github/Yuriigutov/Calculator/blob/main/English_Language_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # Завантажте kaggle.json з вашого комп'ютера

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andriydyachuk","key":"7a83517e789fb2a8031268d5acb64982"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d alphasis/xgboost-with-context-label-data-acc-99-637
!unzip <dataset-file-name>.zip  # Розпаковка файлу, якщо це необхідно

403 - Forbidden - Permission 'datasets.get' was denied
/bin/bash: line 1: dataset-file-name: No such file or directory


In [ ]:
!pip install xgboost pandas numpy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import gc

# Завантажуємо дані з Excel файлу
df = pd.read_excel('text_normalization_challenge.xlsx')

# Перевіряємо наявні стовпці в датасеті
print(df.columns)

# Підготовка даних
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 100

x_data = []

# Перевіряємо стовпець "Written Expression", оскільки це буде наш вхідний текст
for x in df['Written Expression'].values:  # Якщо стовпець "Written Expression"
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)  # Перетворюємо символи в ASCII значення
    x_data.append(x_row)

# Функція для трансформації даних з контекстом
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

# Трансформація даних
x_data = x_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))

gc.collect()

x_data = np.array(x_data)

# Якщо потрібно працювати з категоріями (наприклад, для подальшого використання в моделі машинного навчання)
# Заміна 'Spoken Form' на відповідну категорію
y_data = pd.factorize(df['Spoken Form'])  # Використовуємо 'Spoken Form' для міток
labels = y_data[1]
y_data = y_data[0]

y_data = np.array(y_data)

# Підсумок
print('Total number of samples:', len(x_data))
print('Use:', max_data_size)

Index(['Written Expression', 'Spoken Form'], dtype='object')
Total number of samples: 59
Use: 100


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.1, random_state=2017)
gc.collect()


0

In [ ]:
import xgboost as xgb

num_class = len(labels)

# Створюємо DMatrix
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

# Параметри для моделі
param = {
    'objective': 'multi:softmax',
    'eta': 0.3,
    'max_depth': 10,
    'silent': 1,
    'nthread': -1,
    'num_class': num_class,
    'eval_metric': 'merror'
}

# Тренування моделі
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20, verbose_eval=10)


[0]	valid-merror:1.00000	train-merror:0.96226
[10]	valid-merror:1.00000	train-merror:0.96226
[20]	valid-merror:1.00000	train-merror:0.96226
[21]	valid-merror:1.00000	train-merror:0.96226


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:07:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
import xgboost as xgb

# Перетворюємо x_data в DMatrix
dvalid = xgb.DMatrix(x_data)

# Прогнозування
pred = model.predict(dvalid)  # Використовуємо DMatrix для прогнозування
pred = [labels[int(x)] for x in pred]  # Перетворюємо числові значення в мітки

# Створення відповідних міток для y_data
y_data_labels = [labels[x] for x in y_data]

# Формуємо результат
df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = [''.join([chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]]) for y in x_data]  # Відновлюємо текст із числових значень
df_pred['predict'] = pred  # Додаємо прогнозовані значення
df_pred['target'] = y_data_labels  # Додаємо справжні мітки

# Збереження результатів у CSV файл з escapechar
df_pred.to_csv('pred.csv', index=False, escapechar='\\')

# Збереження помилок у окремий файл
df_errors = df_pred.loc[df_pred['predict'] != df_pred['target']]  # Фільтруємо помилки
df_errors.to_csv('errors.csv', index=False, escapechar='\\')



In [ ]:
model.save_model('xgb_model')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:07:51] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [ ]:
from google.colab import files
files.download('pred.csv')
files.download('errors.csv')
files.download('xgb_model')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>